<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_OMT_distance_between_dose_distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Install Octave 
---



In [ ]:
%%capture 
!! apt-get update
! apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev wget git libsundials-dev gnuplot x11-apps


In [ ]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/octave-colab/tags \
| grep "name" | tail -n 1 \
| awk '{print "https://github.com/cerr/octave-colab/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o octavecolab.zip $LOCATION

unzip octavecolab.zip -d octavecolab
releaseDir=$(ls octavecolab)
tar xzvf "octavecolab/${releaseDir}/octavecolab.tar.gz"

In [ ]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/octavecolab/bin/octave-cli'
os.environ['PATH'] = '/content/octavecolab/bin:' + os.environ['PATH']

## Install Oct2Py and required packages

In [ ]:
%%capture
!pip3 install octave_kernel
!pip3 install oct2py

%load_ext oct2py.ipython

!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"

## Download CERRx

In [ ]:
%%capture
!git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git

## Sample OMT distance calculations

In [ ]:
%%capture
from oct2py import octave

octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')
octave.addpath(octave.genpath('/content/CERR'))

In [ ]:
#Load sample data
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/' + \
             'lung_ex1_20may03.mat.bz2'

%octave_push sampleData

planC = octave.loadPlanC(sampleData,octave.tempdir());
planC = octave.updatePlanFields(planC);
planC = octave.quality_assure_planC(sampleData,planC);

In [ ]:
# Example 1. Sanity check (distance between identical distributions)

doseNum1 = 1;
doseNum2 = 1;
structNum = 3;       #GTV
gamma = 0.1;
downsampleIndex = 3;

dist = octave.calcOMTDoseDistance(doseNum1,doseNum2,structNum,gamma,downsampleIndex,planC);
print('OMT distance = ' + str(dist))

In [ ]:
# Example 2. Distance between two dose distributions in region of interest

doseNum1 = 1;
doseNum2 = 2;
structNum = 3; 
gamma = 0.1;
downsampleIndex = 3;

dist = octave.calcOMTDoseDistance(doseNum1,doseNum2,structNum,gamma,downsampleIndex,planC);
print('OMT distance = ' + str(dist))

## Display ROI and dose distributions

In [ ]:
from oct2py import octave
%octave_push planC structNum doseNum1 doseNum2 dist

In [ ]:
%%octave 

addpath(genpath('/content/CERR'));

# Get scan array
indexS = planC{end};
scanNum = getStructureAssociatedScan(structNum,planC);
scan3M = getScanArray(scanNum,planC);
CToffset = planC{indexS.scan}(1).scanInfo(1).CTOffset;
scan3M = double(scan3M - CToffset);

# Get dose arrays on CT grid
dose1M = getDoseOnCT(doseNum1, scanNum, 'normal', planC);
dose2M = getDoseOnCT(doseNum2, scanNum, 'normal', planC);

# Crop to slices of interest
mask3M = getStrMask(structNum,planC);
[rMin,rMax,cMin,cMax,sMin,sMax] = compute_boundingbox(mask3M);
scan3M = scan3M(:,:,sMin:sMax);
mask3M = mask3M(:,:,sMin:sMax);
dose1M = dose1M(:,:,sMin:sMax);
dose2M = dose2M(:,:,sMin:sMax);
doseC = {dose1M,dose2M};

In [ ]:
%octave_pull doseNum1 doseNum2 structNum doseC scan3M mask3M dist

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
import numpy as np
from matplotlib import cm
from IPython.display import clear_output
from functools import partial
from skimage import measure

clear_output(wait=True)    

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def show_roi(ax,scanM,maskM):
    #Show region of interest
    window_center = 0
    window_width = 300
    extent = 0, 511, 0, 511
   
    im0 = ax.imshow(scanM, cmap=plt.cm.gray, alpha=1,
                    interpolation='none', extent=extent)
    
    contours = measure.find_contours(maskM, 0.5)
    for contour in contours:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='red')

    ax.set_xticks([])
    ax.set_yticks([])
    ax.title.set_text('Region of interest')    

def show_dose_overlay(scan3M, mask3M, doseNum1, doseNum2, dist, slcNum):

    clear_output(wait=True)
    print('Slice '+str(slcNum))
    if 'fig' in locals():
        fig.remove()
   
    # Define color maps & window levels
    window_center = 0
    window_width = 300
    jet=plt.cm.jet
    jet_map = [jet(i) for i in range(jet.N)]
    #jet_map[0] = [0.0,0.0,0.0,1.0]
    dose_cmap = jet.from_list('Custom cmap', jet_map, jet.N)
    dose_cmap.set_under('k', alpha=0)
    
    fig, ax = plt.subplots(1,3)
    fig.set_size_inches(10, 6)
    extent = 0, 511, 0, 511

    # Show ROI
    windowed_img = window_image(scan3M[:,:,slcNum-1],
                                window_center,window_width)
    maskM = mask3M[:,:,slcNum-1]
    show_roi(ax[0],windowed_img,maskM)

    # Show dose1 overlay
    im1 = ax[1].imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='none', extent=extent)
    dose1M = doseC[0][doseNum1-1]
    d1 = ax[1].imshow(dose1M[:,:,slcNum-1], 
                        cmap=dose_cmap, alpha=.4, extent=extent,
                        interpolation='none')  
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].title.set_text('Dose 1')

    # Show dose2 overlay
    im2 = ax[2].imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='none', extent=extent)
    
    dose2M = doseC[0][doseNum2-1];
    d2 = ax[2].imshow(dose2M[:,:,slcNum-1], 
                        cmap=dose_cmap, alpha=.4, extent=extent,
                        interpolation='none',clim=[0,90])  
    
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    ax[2].title.set_text('Dose 2')

    cax = fig.add_axes([0.95,0.32,0.03,0.36]) 
    fig.colorbar(d2, cax=cax)
    fig.subplots_adjust(wspace=0.3)
    txt = 'OMT distance = ' + str("{:.4f}".format(dist))
    fig.text(.66, .25, txt, fontsize = 12, fontweight = 'bold', ha='center')
    
    plt.show()   

slice_slider = widgets.IntSlider(min=1,max=20,step=1)
outputSlc = widgets.Output()
display(slice_slider, outputSlc) 

def update_display(change):
  global scan3M, mask3M, doseNum1, doseNum2, dist
  with outputSlc:
    show_dose_overlay(scan3M, mask3M, int(doseNum1), int(doseNum2), \
                      dist, change['new'])

slice_slider.observe(update_display, names='value')

IntSlider(value=1, max=20, min=1)

Output()